### 1. 安装HuggingFace 并下载模型到本地

In [1]:
!pip install huggingface-hub -Uqq
!pip install -U sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 790.6/790.6 kB 8.2 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 2.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.8/661.8 kB 6.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 23.9 MB/s eta 0:00:0000:010:01m
  Created wheel for sagemaker: filename=sagemaker-2.157.0-py2.py3-none-any.whl size=1066763 sha256=6a856d7b09de5ac3bd0ceb103ae4182f918abf4554a29d62e5d48eade12a4ffe
  Stored in directory: /home/ec2-user/.cache/pip/wheels/cc/95/ca/eefe1e1830c2b049a1c7836ffdaa54a98d152251a2e976892c
Successfully built sagemaker
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.4.1
    Uninstalling PyYAML-5.4.1:
      Successfully uninstall

In [2]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./e5-large-model")
local_model_path.mkdir(exist_ok=True)
model_name = "intfloat/multilingual-e5-large"
commit_hash = "c505dce3578a12ec54e47bdc72bef5cd0eacb085"

In [3]:
snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)

'e5-large-model/models--intfloat--multilingual-e5-large/snapshots/c505dce3578a12ec54e47bdc72bef5cd0eacb085'

### 2. 把模型拷贝到S3为后续部署做准备

In [4]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [5]:
s3_model_prefix = "LLM-RAG/workshop/e5-large-model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/e5_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/e5_deploy_code
model_snapshot_path: e5-large-model/models--intfloat--multilingual-e5-large/snapshots/c505dce3578a12ec54e47bdc72bef5cd0eacb085


In [6]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: e5-large-model/models--intfloat--multilingual-e5-large/snapshots/c505dce3578a12ec54e47bdc72bef5cd0eacb085/1_Pooling/config.json to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/e5-large-model/1_Pooling/config.json
upload: e5-large-model/models--intfloat--multilingual-e5-large/snapshots/c505dce3578a12ec54e47bdc72bef5cd0eacb085/.gitattributes to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/e5-large-model/.gitattributes
upload: e5-large-model/models--intfloat--multilingual-e5-large/snapshots/c505dce3578a12ec54e47bdc72bef5cd0eacb085/config.json to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/e5-large-model/config.json
upload: e5-large-model/models--intfloat--multilingual-e5-large/snapshots/c505dce3578a12ec54e47bdc72bef5cd0eacb085/modules.json to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/e5-large-model/modules.json
upload: e5-large-model/models--intfloat--multilingual-e5-large/snapshots/c505dce3578a12ec54e47bdc72bef5cd0eacb085/README.md to s

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [7]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118"
)

#中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117"
# )

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


In [8]:
!mkdir -p e5_deploy_code

In [9]:
%%writefile e5_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os
# from FlagEmbedding import FlagModel
from sentence_transformers import SentenceTransformer


def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")

    # model =  FlagModel(model_location)
    model = SentenceTransformer(model_location)
    
    return model

model = None

def handle(inputs: Input):
    global model
    if not model:
        model = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = None
    inputs = data["inputs"]
    if isinstance(inputs, list):
        input_sentences = inputs
    else:
        input_sentences =  [inputs]
        
    logging.info(f"inputs: {input_sentences}")

    sentence_embeddings =  model.encode(input_sentences,normalize_embeddings=True)
        
    result = {"sentence_embeddings": sentence_embeddings}
    return Output().add_as_json(result)

Writing e5_deploy_code/model.py


#### Note: option.s3url 需要按照自己的账号进行修改

In [10]:
%%writefile e5_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.s3url = s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/e5-large-model/

Writing e5_deploy_code/serving.properties


In [19]:
%%writefile e5_deploy_code/requirements.txt
transformers==4.30.2
sentence-transformers

Writing e5_deploy_code/requirements.txt


In [20]:
!rm s2e_model.tar.gz
!cd e5_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf s2e_model.tar.gz e5_deploy_code

e5_deploy_code/
e5_deploy_code/model.py
e5_deploy_code/requirements.txt
e5_deploy_code/serving.properties


In [21]:
s3_code_artifact = sess.upload_data("s2e_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/e5_deploy_code/s2e_model.tar.gz


### 4. 创建模型 & 创建endpoint

In [22]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base("e5-mutiling-large") ##Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

e5-mutiling-large-2023-08-18-02-45-51-123
Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118
Created Model: arn:aws:sagemaker:us-east-2:946277762357:model/e5-mutiling-large-2023-08-18-02-45-51-123


In [23]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 10*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:946277762357:endpoint-config/e5-mutiling-large-2023-08-18-02-45-51-123-config',
 'ResponseMetadata': {'RequestId': 'dd3acae7-12fc-4ba6-93db-c80a5f88b49c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dd3acae7-12fc-4ba6-93db-c80a5f88b49c',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '129',
   'date': 'Fri, 18 Aug 2023 02:45:54 GMT'},
  'RetryAttempts': 0}}

In [24]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-2:946277762357:endpoint/e5-mutiling-large-2023-08-18-02-45-51-123-endpoint


In [25]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-2:946277762357:endpoint/e5-mutiling-large-2023-08-18-02-45-51-123-endpoint
Status: InService


### 5. 模型测试

In [26]:
def get_vector_by_sm_endpoint(questions, sm_client, endpoint_name):

    response_model = sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(
            {
                "inputs": questions,
            }
        ),
        ContentType="application/json",
    )
    json_str = response_model['Body'].read().decode('utf8')
    json_obj = json.loads(json_str)
    embeddings = json_obj['sentence_embeddings']
    return embeddings

In [27]:
prompts1 = """请问AWS Clean Rooms是多方都会收费吗？"""
ret = get_vector_by_sm_endpoint(prompts1, smr_client, endpoint_name)

In [28]:
print(len(ret[0]))

1024


### 5. 设置autoscaling(optional)

In [ ]:
asg = boto3.client('application-autoscaling')

# Resource type is variant and the unique identifier is the resource ID.
resource_id=f"endpoint/{endpoint_name}/variant/variant1"

# scaling configuration
response = asg.register_scalable_target(
    ServiceNamespace='sagemaker', #
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount', 
    MinCapacity=1,
    MaxCapacity=4
)

In [ ]:
response = asg.put_scaling_policy(
    PolicyName=f'Request-ScalingPolicy-{endpoint_name}',
    ServiceNamespace='sagemaker',
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    PolicyType='TargetTrackingScaling',
    TargetTrackingScalingPolicyConfiguration={
        'TargetValue': 10.0, # Threshold
        'PredefinedMetricSpecification': {
            'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance',
        },
        'ScaleInCooldown': 300, # duration until scale in
        'ScaleOutCooldown': 60 # duration between scale out
    }
)

In [53]:
!aws sagemaker delete-endpoint --endpoint-name {endpoint_name}

In [54]:
!aws sagemaker delete-endpoint-config --endpoint-config-name {endpoint_config_name}

In [ ]:
!aws sagemaker delete-model --model-name  {model_name}